In [1]:
import os
import torch
import cv2
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
import tensorflow as tf
from flask import Flask, request, jsonify, render_template, Response
from tensorflow.keras.applications.resnet50 import preprocess_input

app = Flask(__name__)
tokenizer = T5Tokenizer.from_pretrained('final_t5_chatbot_tokenizer')
model = T5ForConditionalGeneration.from_pretrained('final_t5_chatbot_model')
# Load emotion recognition model
emotion_model = tf.keras.models.load_model('final_save.h5')

emotion_labels = ['Angry', 'Fear', 'Happy', 'Sad', 'Neutral', 'Surprise']
img_size = (224, 224)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

detected_emotion_global = "Neutral"

initial_dialogues = {
    "Angry": "You seem upset. Is there anything you’d like to talk about?",
    "Fear": "It looks like you're worried. I’m here if you need someone to talk to.",
    "Happy": "You're looking cheerful today! What’s making you feel so good?",
    "Sad": "I’m sorry to see that you’re feeling down. Want to share what’s on your mind?",
    "Neutral": "Hello! How are you feeling today?",
    "Surprise": "You seem surprised! What happened?"
}

def predict_emotion(frame):
    global detected_emotion_global
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    emotions = []
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        img_resized = cv2.resize(face, img_size)
        img_array = np.expand_dims(img_resized, axis=0)
        img_array = preprocess_input(img_array)

        predictions = emotion_model.predict(img_array)
        emotion_index = np.argmax(predictions)
        emotion = emotion_labels[emotion_index]
        emotions.append((emotion, (x, y, w, h)))

    detected_emotion_global = emotions[0][0] if emotions else "Neutral"
    return emotions

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    cap = cv2.VideoCapture(0)

    def generate_frames():
        while True:
            success, frame = cap.read()
            if not success:
                break

            emotions = predict_emotion(frame)
            for emotion, (x, y, w, h) in emotions:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()

            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/detected_emotion', methods=['GET'])
def detected_emotion():
    return jsonify({'emotion': detected_emotion_global})

@app.route('/generate_response', methods=['POST'])
def generate_response():
    data = request.get_json()
    emotion = data.get('emotion', 'Neutral')
    user_input = data.get('userInput', '')

    # Check if user_input is empty and respond accordingly
    if user_input.strip() == '':
        bot_response = initial_dialogues.get(emotion, "Hello! How can I assist you today?")
    else:
        # Prepare input for the T5 model
        input_text = f"{emotion}: {user_input}"
        input_ids = tokenizer.encode(input_text, return_tensors="pt")

        # Generate response from the model
        with torch.no_grad():  # Disable gradient calculation for inference
            output_ids = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)

        # Decode the output to get the response
        bot_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return jsonify({'response': bot_response})

if __name__ == '__main__':
    app.run(debug=False)


c:\Users\bhara\anaconda3\envs\bharath\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Oct/2024 11:45:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:18] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:18] "POST /generate_response HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:23] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 11:45:23] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:28] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:32] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:33] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:34] "POST /generate_response HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:38] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:39] "GET /detected_emotion HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 11:45:39] "POST /generate_r